In [3]:
def calc_c_tdresolved(z,T=25,S=35):
# returns time-depth resolved c, sound speed in m/s
# note: z must be time-depth resolved
    c = 1448.96 + 4.591*T - 0.05304*T**2 + (2.374*10**-4)*T**3 \
        + 1.34*(S-35) + 0.0163*z + (1.675*10**-7)*z**2 - 0.01025*T*(S-35) \
        - (7.139*10**-13)*T*z**3
    return c

In [3]:
def calc_alpha_tdresolved(row,z,c,T,S=35,pH=8.1):
# returns time-depth alpha, absorption coefficient in dB/m
# note: z, c, T must be time-depth resolved
    A1 = (8.86/c)*10**(0.78*pH-5)
    f1 = 2.8*(S/35)**0.5 * 10**(4-1245/(T+273))
    P1 = 1
    A2 = 21.44*S/c*(1+0.025*T)
    P2 = 1 - (1.37*10**-4)*z + (6.2*10**-9)*z**2
    f2 = 8.17*10**(8-1990/(T+273)) / (1 + 0.0018*(S-35))
    P3 = 1 - (3.83*10**-5)*z + (4.9*10**-10)*z**2
    Tle20 = T.where(T<=20)
    A3_Tle20 = 4.937*10**-4 - (2.59*10**-5)*Tle20 + (9.11*10**-7)*Tle20**2 - (1.5*10**-8)*Tle20**3
    Tgt20 = T.where(T>20)
    A3_Tgt20 = 3.964*10**-4 - (1.146*10**-5)*Tgt20 + (1.45*10**-7)*Tgt20**2 - (6.5*10**-10)*Tgt20**3
    A3 = T * np.nan
    A3 = xr.where(T<=20, A3_Tle20, A3)
    A3 = xr.where(T>20, A3_Tgt20, A3)
    f = row['freq_round_kHz']
    alpha = A1*P1*f1*f**2 / (f**2 + f1**2) \
            + A2*P2*f2*f**2 / (f**2 + f2**2) \
            + A3*P3*f**2 # dB/km
    return alpha/1000 #dB/m

In [3]:
def get_C_constant(row):
# returns constant C in dB
    if row['bandwidth']=='narrowband':
        Cnow = 'C_6%'
    elif row['bandwidth']=='broadband':
        Cnow = 'C_25%'
        
    if 'OS' in row['instrument_name']:
        values_tablenow = OS_C_PDBW_Kc_Er_values
        if 'OS-II' in row['instrument_name']:
            instrument_prefixnow = 'OS-II'
        else:
            instrument_prefixnow = 'OS'
    elif 'WH' in row['instrument_name']:
        values_tablenow = WH_C_PDBW_Kc_Er_values
        if 'WH-Mariner' in row['instrument_name']:
            instrument_prefixnow = 'WH-Mariner'
        else:
            instrument_prefixnow = 'WH'
    elif 'BB' in row['instrument_name']:
        values_tablenow = BB_C_PDBW_Kc_Er_values
        instrument_prefixnow = 'BB-VM'
        
    return values_tablenow[
        (values_tablenow['instrument_prefix']==instrument_prefixnow)
        & (values_tablenow['freq_round_kHz']==row['freq_round_kHz'])
        ][Cnow].values
    
def get_C_tdresolved(row,c):
# returns time-depth resolved C in dB
    if 'NB' in row['instrument_name']:
        if 'NB-VM' in row['instrument_name']:
            instrument_prefixnow = 'NB-VM'
        elif 'NB-DR' in row['instrument_name']:
            instrument_prefixnow = 'NB-DR'
        K2 = NB_K2_values[(NB_K2_values['instrument_prefix']==instrument_prefixnow)
                          & (NB_K2_values['freq_round_kHz']==row['freq_round_kHz'])
                         ]['K2'].values
        Ks = NB_Ks_values[(NB_Ks_values['freq_round_kHz']==row['freq_round_kHz'])
                         ]['Ks'].values
        C = 10*np.log10(4.47*10**(-20)*K2*Ks/c)
        return C

In [4]:
def get_R_tdresolved(row, depth, method_num=5, c_wtavg=1475.1): # m
# returns time-depth resolved R in m
    if method_num==4:
        return depth
    if method_num==5:
        return depth/np.cos(row['theta']*np.pi/180)
    if method_num==6:
        # c_wtavg can be time-depth resolved, just like depth
        return depth/np.cos(row['theta']*np.pi/180)*c_wtavg/1475.1

In [5]:
def get_LDBM_constant(row): 
# returns constant LDBM in dB 
# TO DO: eventually write a tdresolved version of this fxn
    return np.log10(row['tpl'])

In [6]:
def get_PDBW_constant(row): 
# returns constant PDBW in dB 
# assumes power supply rather than battery
# assumes 220V voltage supply level for NB-VM and NB-DR --> NO REASON FOR THIS!
# TO DO: maybe eventually write a tdresolved version of this fxn
    if 'NB' not in row['instrument_name']:
        #if row['power_type']=='battery':
        #    PDBWnow = 'PDBW_battery'
        #elif row['power_type']=='power-supply':
        #    PDBWnow = 'PDBW_power_supply'
        PDBWnow = 'PDBW_power_supply' 
        if 'OS' in row['instrument_name']:
            values_tablenow = OS_C_PDBW_Kc_Er_values
            if 'OS-II' in row['instrument_name']:
                instrument_prefixnow = 'OS-II'
            else:
                instrument_prefixnow = 'OS'
        elif 'WH' in row['instrument_name']:
            values_tablenow = WH_C_PDBW_Kc_Er_values
            if 'WH-Mariner' in row['instrument_name']:
                instrument_prefixnow = 'WH-Mariner'
            else:
                instrument_prefixnow = 'WH'
        elif 'BB' in row['instrument_name']:
            values_tablenow = BB_C_PDBW_Kc_Er_values
            instrument_prefixnow = 'BB-VM'
        return values_tablenow[
            (values_tablenow['instrument_prefix']==instrument_prefixnow)
            & (values_tablenow['freq_round_kHz']==row['freq_round_kHz'])
            ][PDBWnow].values

    elif 'NB' in row['instrument_name']:
        # - Check diff btwn assuming 110V and 220V voltage supply level
        # w/ NB-VM instrument, 150kHz freq
        # K1_110 = np.log10(((((110*1.397)-4.27)/37.14)**2)*3.3) 
        # K1_220 = np.log10(((((220*0.699)-4.27)/37.14)**2)*3.3)
        # print(K1_220-K1_110)
        # # --> 0.0006 = miniscule difference

        #voltage_now = row['voltage']
        voltage_now = 220
        if 'NB-VM' in row['instrument_name']:
            instrument_prefix_voltagenow = 'NB-VM-' + str(voltage_now)
            instrument_prefixnow = 'NB-VM'
        elif 'NB-DR' in row['instrument_name']:
            instrument_prefix_voltagenow = 'NB-DR'
            instrument_prefixnow = 'NB-DR'
        K1c = NB_K1c_values[
            (NB_K1c_values['instrument_prefix']==instrument_prefixnow)
            & (NB_K1c_values['freq_round_kHz']==row['freq_round_kHz'])
            ]['K1c'].values
        a = NB_abc_values[
            (NB_abc_values['instrument_prefix_voltage']==instrument_prefix_voltagenow)
            ]['a'].values
        b = NB_abc_values[
            (NB_abc_values['instrument_prefix_voltage']==instrument_prefix_voltagenow)
            ]['b'].values
        c = NB_abc_values[
            (NB_abc_values['instrument_prefix_voltage']==instrument_prefix_voltagenow)
            ]['c'].values
        K1 = (((voltage_now * a) - b)/c)**2 * K1c
        return np.log10(K1)

In [7]:
def calc_alphaR_tdresolved(alpha,R):
# returns time-depth resolved alpha*R in dB
# note: alpha, R must be time-depth resolved
    dR = R*np.nan
    dR[:,0] = R[:,0]
    dR[:,1:] = R.diff('depth_cell')
    alpha_dR1 = alpha*dR
    alpha_dR = alpha_dR1*np.nan
    alpha_dR[:,0] = alpha_dR1[:,0]
    for idepth in np.arange(1,alpha_dR.depth_cell.size):
        alpha_dR[:,idepth] = alpha_dR1[:,0:(idepth+1)].sum(dim='depth_cell')
    return alpha_dR

In [4]:
def get_Kc_constant(row):
# returns constant Kc in dB/count
    if 'OS' in row['instrument_name']:
        values_tablenow = OS_C_PDBW_Kc_Er_values
        if 'OS-II' in row['instrument_name']:
            instrument_prefixnow = 'OS-II'
        else:
            instrument_prefixnow = 'OS'
    elif 'WH' in row['instrument_name']:
        values_tablenow = WH_C_PDBW_Kc_Er_values
        if 'WH-Mariner' in row['instrument_name']:
            instrument_prefixnow = 'WH-Mariner'
        else:
            instrument_prefixnow = 'WH'
    elif 'BB' in row['instrument_name']:
        values_tablenow = BB_C_PDBW_Kc_Er_values
        instrument_prefixnow = 'BB-VM'
    return values_tablenow[
        (values_tablenow['instrument_prefix']==instrument_prefixnow)
        & (values_tablenow['freq_round_kHz']==row['freq_round_kHz'])
        ]['Kc_mean'].values

def get_Kc_tdresolved(row,amp,Tx):
# returns time-depth resolved Kc in dB/count
# this assumes that Tx = temp of transducer = Te = temp of system electronics
    if 'NB' in row['instrument_name']:
        Kc_amplt200 = (127.3/(Tx+271)).broadcast_like(amp)
        Kc = amp * np.nan
        Kc = xr.where(amp<200, Kc_amplt200, Kc) 
        return Kc

In [9]:
def get_Er_constant(row,amp):
# returns constant Er in counts
    return amp.min()

In [5]:
# https://stackoverflow.com/questions/13314626/local-solar-time-function-from-utc-and-longitude
# https://www.esrl.noaa.gov/gmd/grad/solcalc/solareqns.PDF
def get_solar_time_and_angles(dt,longit,latit): 
    gamma = 2 * pi / 365 * (dt.timetuple().tm_yday - 1 + float(dt.hour - 12) / 24) # unit = radians
    eqtime = 229.18 * (0.000075 + 0.001868 * cos(gamma) - 0.032077 * sin(gamma) \
             - 0.014615 * cos(2 * gamma) - 0.040849 * sin(2 * gamma)) # unit = minutes
    decl = 0.006918 - 0.399912 * cos(gamma) + 0.070257 * sin(gamma) \
           - 0.006758 * cos(2 * gamma) + 0.000907 * sin(2 * gamma) \
           - 0.002697 * cos(3 * gamma) + 0.00148 * sin(3 * gamma) # unit = radians
    time_offset = eqtime + 4 * longit # unit = minutes
    tst = dt.hour * 60 + dt.minute + dt.second / 60 + time_offset # unit = minutes
    solar_time = datetime.combine(dt.date(), time(0)) + timedelta(minutes=tst) # unit = datetime
    solar_hour_angle = (tst/4) - 180 # unit = degrees
    solar_zenith_angle = 180/pi * acos(sin(latit*pi/180)*sin(decl) + \
                                       cos(latit*pi/180)*cos(decl)*cos(solar_hour_angle*pi/180)) # unit = degrees
    solar_alt_angle = 90 - solar_zenith_angle # unit = degrees
    #return solar_time, solar_hour_angle, solar_zenith_angle, solar_alt_angle
    return solar_time, solar_alt_angle